## [Create VPC](https://blog.ipswitch.com/how-to-create-and-configure-an-aws-vpc-with-python)


#### add [boto3 credentials](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html) to ~/.aws/credentials using a profile_name of form:

```
[default]
aws_access_key_id = XXXXXXXXXXXXXXXXXXXXXXXXXXXX
aws_secret_access_key = xxxxxxxxxxxxxxxxxxxxxxxxxxx

region=us-east-1


[seigniorage]
aws_access_key_id = XXXXXXXXXXXXXXXXXXXXXXXXXXX
aws_secret_access_key = xxxxxxxxxxxxxxxxxxxxxxxxx

region=us-east-2
```

In [ ]:
import boto3
import os


session = boto3.Session(profile_name='seigniorage')
ec2 = session.resource('ec2')

## Create Network
1. VPC
2. InternetGateway
3. Route Table
4. Subnets
5. Security Group

In [ ]:
KEY_NAME  =  "blockchain-nodes-keypair"
KEY_DIRECTORY = "/data/code/software/aws/seigniorage/test/"

SECURITYGROUP_NAME  =  "blockchain-nodes-sg"

DATAVOLUME_NAME     =  "Algorand Mainnet Chain Data"
DATAVOLUME_SIZE     =  64

INSTANCE_NAME = 'Algorand Mainnet'
Amazon_Ubuntu_AMI_18_04_LTS = 'ami-0d5d9d301c853a04a'
Instance_Type = 't2.medium' #micro=1GB, small=2GB, medium=4GB, large=8GB https://us-east-2.console.aws.amazon.com/ec2/v2/home?region=us-east-2#LaunchInstanceWizard:

AVZONE = session.region_name+'b'
VPC_ID, SUBNET_ID = get_subnet_id(AVZONE,"public")
SECURITY_GROUP_ID = get_security_group_id(session,VPC_ID,SECURITYGROUP_NAME)

### Create Key Pair for EC2 Instance

1. key pem files can be used multiple times; just create on for all EC2 instances

In [ ]:
file = KEY_DIRECTORY + KEY_NAME + '.pem'
outfile = open(file,'w')

# call the boto ec2 function to create a key pair
key_pair = ec2.create_key_pair(KeyName=KEY_NAME)

# capture the key and store it in a file
KeyPairOut = str(key_pair.key_material)
print(KeyPairOut)
outfile.write(KeyPairOut)
outfile.close()

os.chmod(file,0o400)

### Create Virtual Private Cloud (VPC)

In [ ]:
vpc = ec2.create_vpc(CidrBlock='10.1.0.0/16')

# we can assign a name to vpc, or any resource, by using tag
vpc.create_tags(Tags=[{"Key": "Name", "Value": VPC_NAME}])

vpc.wait_until_available()
print(vpc.id)

### Create and Attach Internet Gateway

In [ ]:
# create then attach internet gateway
ig = ec2.create_internet_gateway()
ig.create_tags(Tags=[{"Key": "Name", "Value": IGW_NAME}])
vpc.attach_internet_gateway(InternetGatewayId=ig.id)
print(ig.id)

### Create a Route Table and a Public Route

In [ ]:
# create a route table and a public route
route_table = vpc.create_route_table()
route_table.create_tags(Tags=[{"Key": "Name", "Value": ROUTETABLE_NAME}])

route = route_table.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=ig.id
)

print(route_table.id)

### Create subnets

1. Why so many?  Do we need both a public-a and public-b subnet?

In [ ]:
# create subnets
# only associate public nets with route table

subnet1 = ec2.create_subnet(CidrBlock='10.1.1.0/24', VpcId=vpc.id, AvailabilityZone=SUBNET+'a')
subnet1.create_tags(Tags=[{"Key": "Name", "Value": SUBNET_NAME + "public-a"}])
# associate the route table with the subnet
route_table.associate_with_subnet(SubnetId=subnet1.id)

subnet2 = ec2.create_subnet(CidrBlock='10.1.2.0/24', VpcId=vpc.id, AvailabilityZone=SUBNET+'b')
subnet2.create_tags(Tags=[{"Key": "Name", "Value": SUBNET_NAME + "public-b"}])
# associate the route table with the subnet
route_table.associate_with_subnet(SubnetId=subnet2.id)

subnet3 = ec2.create_subnet(CidrBlock='10.1.3.0/24', VpcId=vpc.id, AvailabilityZone=SUBNET+'a')
subnet3.create_tags(Tags=[{"Key": "Name", "Value": SUBNET_NAME + "private-a"}])

subnet4 = ec2.create_subnet(CidrBlock='10.1.4.0/24', VpcId=vpc.id, AvailabilityZone=SUBNET+'b')
subnet4.create_tags(Tags=[{"Key": "Name", "Value": SUBNET_NAME + "private-b"}])

print('subnets',subnet1.id,subnet2.id,subnet3.id,subnet4.id)

### Create Security Group

In [ ]:
# Create security group
sec_group = ec2.create_security_group(
    GroupName=SECURITYGROUP_NAME, Description='Allow SSH, HTTP and HTTPS', VpcId=vpc.id)

sec_group.authorize_ingress(
    GroupId=sec_group.id,
    IpPermissions=[
        {'IpProtocol': 'tcp',
         'FromPort': 80,
         'ToPort': 80,
         'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        {'IpProtocol': 'tcp',
         'FromPort': 443,
         'ToPort': 443,
         'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        {'IpProtocol': 'tcp',
         'FromPort': 8080,
         'ToPort': 8080,
         'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        {'IpProtocol': 'tcp',
         'FromPort': 22,
         'ToPort': 22,
         'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
    ])

sec_group.create_tags(Tags=[{"Key": "Name", "Value": SECURITYGROUP_NAME}])

instance_security_group_id=sec_group.id

print(sec_group.id)